# OpenStreetMap Data Wrangling with SQL

#### Report Environment

* Python 3
* SQLite

## Map Area

**Location:** Istanbul, Turkey

* [OpenStreetMap URL](https://www.openstreetmap.org/node/1882099475)
* [MapZen URL](https://mapzen.com/data/metro-extracts/metro/istanbul_turkey/)

If there are some places I would like to visit or visit again, Istanbul is definitely on the list. From splendour Byzantine to powerful Ottoman Empire, Istanbul is full of mystery and attraction. That's why I chose this map, and I’d like an opportunity to contribute to its improvement on OpenStreetMap.org.

This data is the suggested Metro Extract with overall filesize of 270 MB (uncompressed).

In [1]:
from audit import *

# declare data
datadir = './Data/'
fname = 'istanbul_turkey.osm'
OSM = datadir + fname

## Data Audit

In [2]:
# count the elements of different tags
count_tags(OSM)


OSM tag: 1;
Node tag: 1245825;
Way tag: 208984;
Relaton tag: 763
    


In [3]:
# audit keys of tag type elements
audit_keys(OSM, more=False)


Keys of tag type elements:
375578 keys with only alpha characters and '_';
31013 keys with alpha characters, '_' and a colon;
511 keys with alpha characters, '_' and two colons;
5 keys with problem characters.
285 keys with other patterns.
Assign argument more=True to see more.
    


In [4]:
# audit street types which are not expected (show only top 5 results)
"""
Three common street types of Turkey are:
* Bulvarı - Boulevard
* Caddesi - Street
* Sokak - Street
"""
audit_street_types(OSM)

sokak: 64 items
Sk.: 63 items
Sokağı: 63 items
Cad.: 54 items
caddesi: 48 items


In [5]:
# audit postcodes
"""
Postal codes in Turkey consist of 5 digits.
Starting with the 2 digit license plate code of the provinces
followed by three digits to specify the location within it.
Istanbul code: 34 (41, 81 are also possible in this map)
"""
audit_postcodes(OSM)

Invalid postcode: 30464
Invalid postcode: 364303
Invalid postcode: 33400
Invalid postcode: 33400
Invalid postcode: 80650


In [6]:
# audit phone numbers
"""
Turkey local phone numbers: seven digits (3+4)
Turkey country calling code: +90 
Istanbul area codes (European side): 212
Istanbul area codes (Asian side): 216
But this map may cover areas other than these 2,
so we do not validate the area code in the project.
"""
audit_phone_numbers(OSM, more=False)


There are 3848 phone numbers in this map:
48 numbers are not actually numbers;
91 numbers are missing country code;
27 numbers are missing area code;
28 numbers are missing both country code and area code;
488 numbers are not well formated
3081 numbers have other problems (e.g. extra zeros, 
wrong country code, more than one phone numbers etc.)
Assign argument more=True to see more.
    


## Problems Encountered in the Map

* Over­abbreviated street names (e.g. "Şemsettin Günaltay Cd.")
* Incorrect tag key (e.g. ```<tag k="addr:street" v="http://www.istiklalsthouse.com/"/>```)
* Inconsistent phone numbers (e.g. "0212 219 55 18")
* Incorrect phone number country code (e.g. "+92122937531")
* Incorrect postcode (e.g. "364303")

### Over­abbreviated street names
Spell out all street types.

In [7]:
# test to clean street names
street_types = {
    'Sk.': {'İnce Yol Sk.', 'İsmailpaşa Sk.'},
    'sokak': {'basakci sokak', 'Cümbüş sokak'},
    'Cad.': {'Emirhan Cad.', 'Atatürk Cad.'},
    'Sokağı': {'Katip Sinan Camii Sokağı', 'Şişhane Sokağı'},
    'Sok.': {'Behzat Budak Sok.', '44 Sok.'}
}
clean_street_names(street_types)

İsmailpaşa Sk. => İsmailpaşa Sokak
İnce Yol Sk. => İnce Yol Sokak
Cümbüş sokak => Cümbüş Sokak
basakci sokak => basakci Sokak
Atatürk Cad. => Atatürk Caddesi
Emirhan Cad. => Emirhan Caddesi
Katip Sinan Camii Sokağı => Katip Sinan Camii Sokak
Şişhane Sokağı => Şişhane Sokak
Behzat Budak Sok. => Behzat Budak Sokak
44 Sok. => 44 Sokak


### Inconsistent phone number format and incorrect country code
Fix incorrect cpuntry code and convert, where necessary, to [E.123](https://en.wikipedia.org/wiki/E.123) pattern.

In [8]:
# test to clean phone numbers
phone_numbers = {
    '(216) 524 15 70',
    '00 90 212 5179393',
    '0216 417 62 10',
    '+90 (0) 212 244 44 99',
    '+91 212 361 6096',
    '+92122937531'
}
clean_phone_numbers(phone_numbers)

+91 212 361 6096 => +90 212 361 6096
00 90 212 5179393 => +90 212 517 9393
+92122937531 => +90 212 293 7531
0216 417 62 10 => +90 216 417 6210
+90 (0) 212 244 44 99 => +90 212 244 4499
(216) 524 15 70 => +90 216 524 1570


## Data Overview

This section contains basic statistics about the Istanbul OpenStreetMap dataset and the SQL queries used to gather them.

### File sizes

```
istanbul_turkey.osm...............270Mb

nodes.csv.........................102Mb
nodes_tags.csv......................3Mb
ways.csv...........................12Mb
ways_nodes.csv.....................38Mb
ways_tags.csv......................11Mb

IstanbulMap.db....................283Mb
```

#### Tables in IstanbulMap.db
```
Node....................1231292 records
NodeTag...................79720 records
Way......................205829 records
WayTag...................317573 records
WayNode.................1589788 records
User.......................2412 records
```

### Religions

```SQL
SELECT u.Value, count(*) as num 
FROM (
    SELECT * FROM NodeTag WHERE Value
    UNION ALL
    SELECT * FROM WayTag) u
WHERE u.Key = 'religion' 
GROUP BY u.Value 
ORDER BY num DESC;
```
result:
```
muslim|1971
christian|101
jewish|10
```

#### Muslim sects

```SQL
SELECT WayTag.Value, count(*) as num
FROM WayTag JOIN (
    SELECT *
    FROM WayTag
    WHERE Key = 'religion' and Value = 'muslim') j
    ON WayTag.WayId = j.WayId
WHERE WayTag.Key = 'denomination'
GROUP BY WayTag.Value
ORDER BY num DESC;
```

result

```
sunni|439
alevi|4
shia|1
```

#### Christian denominations

```SQL
SELECT WayTag.Value, count(*) as num
FROM WayTag JOIN (
    SELECT *
    FROM WayTag
    WHERE Key = 'religion' and Value = 'christian') j
    ON WayTag.WayId = j.WayId
WHERE WayTag.Key = 'denomination'
GROUP BY WayTag.Value
ORDER BY num DESC;
```

result

```
orthodox|18
greek_orthodox|13
catholic|7
protestant|4
anglican|1
bulgarian_orthodox|1
new_apostolic|1
palestinian_orthodox|1
presbyterian|1
roman_catholic|1
```

### History and culture

#### Historic
```SQL
SELECT u.Value, count(*) as num 
FROM (
    SELECT * FROM NodeTag
    UNION ALL
    SELECT * FROM WayTag) u
WHERE u.Key = 'historic' 
GROUP BY u.Value 
ORDER BY num DESC;
```
result:
```
ruins|81
memorial|53
monument|48
castle|21
citywalls|19
archaeological_site|11
tomb|5
heritage|3
yes|2
church|1
city_gate|1
fort|1
wayside_shrine|1
```

#### Byzantine
```SQL
SELECT WayTag.Value, count(*) as num 
FROM WayTag JOIN (
    SELECT * 
    FROM WayTag
    WHERE Value = 'byzantine') j
    ON WayTag.WayId = j.WayId
WHERE WayTag.Key = 'name'
GROUP BY WayTag.WayId
ORDER BY num DESC;
```
result:
```
Aya Sofya|1
Yenikapı|1
```

### Tourism

#### Top 20 Popular Places for Tourists
```SQL
SELECT WayTag.Value, count(*) as num 
FROM WayTag JOIN (
    SELECT * 
    FROM WayTag
    WHERE Type = 'name' or Type = 'alt_name' or Key = 'name' or Key = 'alt_name') j
    ON WayTag.WayId = j.WayId
WHERE WayTag.Type = 'name' and WayTag.Key = 'en'
GROUP BY WayTag.WayId
ORDER BY num DESC
LIMIT 20;
```
result:
```
Bosphorus|67
Hagia Sophia|17
Fatih Sultan Mehmet Bridge|15
Galata Tower|10
Crimea Memorial Church|10
Topkapı Palace|9
Süleymaniye Mosque|8
Istanbul Atatürk Airport|8
Church of St. George|8
Egyptian (Spice) Bazaar|7
Kariye Museum|7
Church of Saint Anthony of Padua|7
Molla Zeyrek Mosque Монастырь Пантократора ( Христа Вседержителя )|7
Basilica Cistern|7
Valens Aqueduct|6
Sultan Ahmet Square|6
Grand Bazaar|6
Market square|6
Court house|6
Sehzade Camii Prince's Mosque|5
```

### Urban life

#### Top 10 Common Amenities
```SQL
SELECT u.Value, count(*) as num 
FROM (
    SELECT * FROM NodeTag
    UNION ALL
    SELECT * FROM WayTag) u
WHERE u.Key = 'amenity' 
GROUP BY u.Value 
ORDER BY num DESC 
LIMIT 10;
```
result:
```
pharmacy|2614
place_of_worship|2299
parking|1300
restaurant|1168
cafe|949
school|906
fuel|622
bank|545
fast_food|347
atm|284
```

### Cuisine

```SQL
SELECT a.value, count(*) as num
FROM (
    SELECT * FROM NodeTag
    UNION ALL
    SELECT * FROM WayTag) a
    JOIN (
    SELECT distinct(NodeId)
    FROM NodeTag
    WHERE NodeTag.NodeId NOT IN (
        SELECT WayNode.NodeId
        FROM WayNode)) b
    ON a.NodeId = b.NodeId
WHERE a.Key = 'cuisine'
GROUP BY a.Value
ORDER BY num DESC
LIMIT 10;
```
result:
```
turkish|140
burger|55
coffee_shop|50
regional|39
kebab|32
pizza|27
international|11
seafood|10
ice_cream|8
italian|8
```

#### Most popular cofeeshop chains
```SQL
SELECT a.Value, count(*) as num
FROM (
    SELECT * FROM NodeTag
    UNION ALL
    SELECT * FROM WayTag) a
    JOIN (
    SELECT distinct(NodeId) FROM NodeTag
    WHERE NodeTag.Value = 'cafe' and NodeTag.NodeId NOT IN (
        SELECT WayNode.NodeId FROM WayNode
        JOIN (
        SELECT distinct(WayId) FROM WayTag
        WHERE WayTag.Value = 'cafe') b
        ON WayNode.WayId = b.WayId)
        UNION ALL
        SELECT distinct(WayId) FROM WayTag
        WHERE WayTag.Value='cafe') c
    ON a.NodeId = c.NodeId
WHERE a.key = 'name'
GROUP BY a.Value
ORDER BY num DESC
LIMIT 10;
```
result:
```
Starbucks|38
Mado|11
Kahve Dünyası|10
Cafeteria|9
Simit Sarayı|5
Caffè Nero|4
Köy kahvesi - Çayocağı|4
Aslı Börek|3
Kahve Diyarı|3
Planet|3
```

## Additional Suggestion and Ideas

The Istanbul OpenStreetMap dataset is a quite messy. An example, let's look back at two queries performed above to perform a new query:

#### Building values
```SQL
SELECT u.Value, count(*) as num 
FROM (
    SELECT * FROM NodeTag
    UNION ALL
    SELECT * FROM WayTag) u
WHERE u.Key = 'building' 
GROUP BY u.Value 
ORDER BY num DESC 
LIMIT 10;
```
result:
```
yes|44846
apartments|5745
residential|4303
house|3569
commercial|972
school|527
roof|412
mosque|399
industrial|380
greenhouse|249
```

#### Total buildings
```SQL
SELECT count(*)
FROM (
    SELECT * FROM NodeTag
    UNION ALL
    SELECT * FROM WayTag) u
WHERE u.Key = 'building';
```
result:
```
62249
```

Among all tags with key 'building', approximately 72% (44846 / 62249) has 'yes' value while 18% has other string value. It is not benefit for analysing data. In fact, this situation also appears in other keys.

My suggestion on improving this is to build a validation system, this might involve:
* Audit data type. Should the value be a booean for the building (yes or no), or should it be a type of building (residential or school).
* Make rules or patterns for special keys. Phone number should be separated into country code, area code and local number (may match E.123 standard). For postcode, the length and the pattern should be unified. And so on.

Benefits:

* It can reduce the chance of typo error.
* It can improve the consistency of data.

Anticipated Problems:

* The rules and patterns should be localized for different country or area.
* It might sacrifice flexibility in a certain degree, and might increase the learning cost for contributors.

#### References

* [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/Main_Page)
* [Postal codes in Turkey @ wikipedia](https://en.wikipedia.org/wiki/Postal_codes_in_Turkey)
* [Telephone numbers in Turkey @ wikipedia](https://en.wikipedia.org/wiki/Telephone_numbers_in_Turkey)